In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib import rc
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt


#맥에서 한글 안깨지게
rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

df = pd.read_csv("../data/train/train.csv")
df_holidays = pd.read_csv("../data/holidays_2023_2025.csv")

# 연, 월, 일 col 생성
df['영업일자'] = pd.to_datetime(df['영업일자'])
df['연'] = df['영업일자'].dt.year
df['월'] = df['영업일자'].dt.month
df['일'] = df['영업일자'].dt.day
# 영업장, 메뉴명 col 생성
df[['영업장', '메뉴명']] = df['영업장명_메뉴명'].str.split('_', expand=True)
# 요일 col 생성
df['요일'] = df['영업일자'].dt.dayofweek.map({
    0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'
})
# 공휴일 col 생성
df_holidays["locdate"] = pd.to_datetime(df_holidays["locdate"])
holiday_dates = set(df_holidays['locdate'])
df['is_holiday'] = df['영업일자'].isin(holiday_dates).astype(int)

# 라그로타, 화담숲 주막 데이터셋
df_lagrota = df.loc[df['영업장'] == "라그로타"]
df_jumak = df.loc[df['영업장'] == "화담숲주막"]
df_mirasia = df.loc[df['영업장'] == "미라시아"]
df_damha = df.loc[df['영업장'] == '담하']

In [2]:
menu_category = {
    '1인 수저세트': '기타', 
    'BBQ55(단체)': '메인메뉴', 
    '대여료 60,000원': '기타',
    '대여료 30,000원': '기타', 
       '대여료 90,000원': '기타', 
       '본삼겹 (단품,실내)': '메인메뉴', 
       '스프라이트 (단체)': '음료', 
       '신라면': '추가메뉴', 
       '쌈야채세트': '추가메뉴', 
       '쌈장': '추가메뉴',
       '육개장 사발면': '추가메뉴', 
       '일회용 소주컵': '기타', 
       '일회용 종이컵': '기타', 
       '잔디그늘집 대여료 (12인석)': '기타',
       '잔디그늘집 대여료 (6인석)': '기타', 
       '잔디그늘집 의자 추가': '기타', 
       '참이슬 (단체)': '주류', 
       '친환경 접시 14cm': '기타',
       '친환경 접시 23cm': '기타', 
       '카스 병(단체)': '주류', 
       '콜라 (단체)': '음료', 
       '햇반': '추가메뉴', 
       '허브솔트': '추가메뉴', 
       '(단체) 공깃밥': '추가메뉴',
       '(단체) 생목살 김치전골 2.0': '메인메뉴', 
       '(단체) 은이버섯 갈비탕': '메인메뉴', 
       '(단체) 한우 우거지 국밥': '메인메뉴',
       '(단체) 황태해장국 3/27까지': '메인메뉴', 
       '(정식) 된장찌개': '메인메뉴', 
       '(정식) 물냉면 ': '메인메뉴', 
       '(정식) 비빔냉면': '메인메뉴',
       '(후식) 된장찌개': '추가메뉴', 
       '(후식) 물냉면': '추가메뉴', 
       '(후식) 비빔냉면': '추가메뉴', 
       '갑오징어 비빔밥': '메인메뉴', 
       '갱시기': '메인메뉴', 
       '공깃밥': '추가메뉴',
       '꼬막 비빔밥': '메인메뉴', 
       '느린마을 막걸리': '주류', 
       '담하 한우 불고기': '메인메뉴', 
       '담하 한우 불고기 정식': '메인메뉴', 
       '더덕 한우 지짐': '메인메뉴',
       '들깨 양지탕': '메인메뉴', 
       '라면사리': '추가메뉴', 
       '룸 이용료': '기타', 
       '메밀면 사리': '추가메뉴', 
       '명인안동소주': '주류', 
       '명태회 비빔냉면': '메인메뉴',
       '문막 복분자 칵테일': '주류', 
       '봉평메밀 물냉면': '메인메뉴', 
       '생목살 김치찌개': '메인메뉴', 
       '스프라이트': '음료', 
       '은이버섯 갈비탕': '메인메뉴', 
       '제로콜라': '음료',
       '참이슬': '주류', 
       '처음처럼': '주류', 
       '카스': '주류', 
       '콜라': '음료', 
       '테라': '주류', 
       '하동 매실 칵테일': '주류', 
       '한우 떡갈비 정식': '메인메뉴',
       '한우 미역국 정식': '메인메뉴', 
       '한우 우거지 국밥': '메인메뉴', 
       '한우 차돌박이 된장찌개': '메인메뉴', 
       '황태해장국': '메인메뉴', 
       'AUS (200g)': '메인메뉴',
       'G-Charge(3)': '기타', 
       'Gls.Sileni': '주류', 
       'Gls.미션 서드': '주류', 
       'Open Food': '기타',
       '그릴드 비프 샐러드': '메인메뉴', 
       '까르보나라': '메인메뉴', 
       '모둠 해산물 플래터': '메인메뉴', 
       '미션 서드 카베르네 쉬라': '메인메뉴', 
       '버섯 크림 리조또': '메인메뉴',
       '빵 추가 (1인)': '추가메뉴', 
       '시저 샐러드 ': '메인메뉴', 
       '아메리카노': '음료', 
       '알리오 에 올리오 ': '메인메뉴', 
       '양갈비 (4ps)': '메인메뉴',
       '자몽리치에이드': '음료', 
       '하이네켄(생)': '주류', 
       '한우 (200g)': '메인메뉴', 
       '해산물 토마토 리조또': '메인메뉴', 
       '해산물 토마토 스튜 파스타': '메인메뉴',
       '해산물 토마토 스파게티': '메인메뉴', 
       '(단체)브런치주중 36,000': '메인메뉴', 
       '(오븐) 하와이안 쉬림프 피자': '메인메뉴',
       '(화덕) 불고기 페퍼로니 반반피자': '메인메뉴', 
       'BBQ Platter': '메인메뉴', 
       'BBQ 고기추가': '추가메뉴', 
       '글라스와인 (레드)': '주류',
       '레인보우칵테일(알코올)': '주류', 
       '미라시아 브런치 (패키지)': '메인메뉴', 
       '버드와이저(무제한)': '주류', 
       '보일링 랍스타 플래터': '메인메뉴',
       '보일링 랍스타 플래터(덜매운맛)': '메인메뉴', 
       '브런치 2인 패키지 ': '메인메뉴', 
       '브런치 4인 패키지 ': '메인메뉴', 
       '브런치(대인) 주말': '메인메뉴',
       '브런치(대인) 주중': '메인메뉴', 
       '브런치(어린이)': '메인메뉴', 
       '쉬림프 투움바 파스타': '메인메뉴', 
       '스텔라(무제한)': '주류', 
       '애플망고 에이드': '음료',
       '얼그레이 하이볼': '주류', 
       '오븐구이 윙과 킬바사소세지': '메인메뉴', 
       '유자 하이볼': '주류', 
       '잭 애플 토닉': '주류', 
       '칠리 치즈 프라이': '추가메뉴',
       '코카콜라': '음료', 
       '코카콜라(제로)': '음료', 
       '콥 샐러드': '추가메뉴', 
       '파스타면 추가(150g)': '추가메뉴', 
       '핑크레몬에이드': '음료',
       'Cass Beer': '주류', 
       'Conference L1': '연회장 대여', 
       'Conference L2': '연회장 대여', 
       'Conference L3': '연회장 대여',
       'Conference M1': '연회장 대여', 
       'Conference M8': '연회장 대여', 
       'Conference M9': '연회장 대여',
       'Convention Hall': '연회장 대여', 
       'Cookie Platter': '디저트', 
       'Grand Ballroom': '연회장 대여', 
       'OPUS 2': '연회장 대여',
       'Regular Coffee': '음료', 
       '골뱅이무침': '메인메뉴', 
       '돈목살 김치찌개 (밥포함)': '메인메뉴', 
       '로제 치즈떡볶이': '메인메뉴', 
       '마라샹궈': '메인메뉴',
       '매콤 무뼈닭발&계란찜': '메인메뉴', 
       '모둠 돈육구이(3인)': '메인메뉴', 
       '삼겹살추가 (200g)': '추가메뉴', 
       '야채추가': '추가메뉴', 
       '왕갈비치킨': '메인메뉴',
       '주먹밥 (2ea)': '추가메뉴', 
       '공깃밥(추가)': '추가메뉴', 
       '구슬아이스크림': '디저트', 
       '단체식 13000(신)': '메인메뉴', 
       '단체식 18000(신)': '메인메뉴',
       '돼지고기 김치찌개': '메인메뉴', 
       '복숭아 아이스티': '음료', 
       '새우 볶음밥': '메인메뉴', 
       '새우튀김 우동': '메인메뉴', 
       '샷 추가': '추가메뉴', 
       '수제 등심 돈까스': '메인메뉴',
       '아메리카노(HOT)': '음료', 
       '아메리카노(ICE)': '음료', 
       '약 고추장 돌솥비빔밥': '메인메뉴', 
       '어린이 돈까스': '메인메뉴', 
       '오픈푸드': '기타',
       '진사골 설렁탕': '메인메뉴', 
       '짜장면': '메인메뉴', 
       '짜장밥': '메인메뉴', 
       '짬뽕': '메인메뉴', 
       '짬뽕밥': '메인메뉴', 
       '치즈돈까스': '메인메뉴', 
       '카페라떼(HOT)': '음료',
       '카페라떼(ICE)': '음료', 
       '한상 삼겹구이 정식(2인) 소요시간 약 15~20분': '메인메뉴', 
       '꼬치어묵': '메인메뉴', 
       '떡볶이': '메인메뉴', 
       '생수': '음료',
       '치즈 핫도그': '디저트', 
       '페스츄리 소시지': '디저트', 
       '단호박 식혜 ': '음료', 
       '병천순대': '메인메뉴', 
       '참살이 막걸리': '주류', 
       '찹쌀식혜': '음료', 
       '해물파전': '메인메뉴',
       '메밀미숫가루': '음료', 
       '아메리카노 HOT': '음료', 
       '아메리카노 ICE': '음료', 
       '카페라떼 ICE': '음료', 
       '현미뻥스크림': '디저트'

}
all_menu = df['메뉴명'].unique()
all_menu

array(['1인 수저세트', 'BBQ55(단체)', '대여료 30,000원', '대여료 60,000원',
       '대여료 90,000원', '본삼겹 (단품,실내)', '스프라이트 (단체)', '신라면', '쌈야채세트', '쌈장',
       '육개장 사발면', '일회용 소주컵', '일회용 종이컵', '잔디그늘집 대여료 (12인석)',
       '잔디그늘집 대여료 (6인석)', '잔디그늘집 의자 추가', '참이슬 (단체)', '친환경 접시 14cm',
       '친환경 접시 23cm', '카스 병(단체)', '콜라 (단체)', '햇반', '허브솔트', '(단체) 공깃밥',
       '(단체) 생목살 김치전골 2.0', '(단체) 은이버섯 갈비탕', '(단체) 한우 우거지 국밥',
       '(단체) 황태해장국 3/27까지', '(정식) 된장찌개', '(정식) 물냉면 ', '(정식) 비빔냉면',
       '(후식) 된장찌개', '(후식) 물냉면', '(후식) 비빔냉면', '갑오징어 비빔밥', '갱시기', '공깃밥',
       '꼬막 비빔밥', '느린마을 막걸리', '담하 한우 불고기', '담하 한우 불고기 정식', '더덕 한우 지짐',
       '들깨 양지탕', '라면사리', '룸 이용료', '메밀면 사리', '명인안동소주', '명태회 비빔냉면',
       '문막 복분자 칵테일', '봉평메밀 물냉면', '생목살 김치찌개', '스프라이트', '은이버섯 갈비탕', '제로콜라',
       '참이슬', '처음처럼', '카스', '콜라', '테라', '하동 매실 칵테일', '한우 떡갈비 정식',
       '한우 미역국 정식', '한우 우거지 국밥', '한우 차돌박이 된장찌개', '황태해장국', 'AUS (200g)',
       'G-Charge(3)', 'Gls.Sileni', 'Gls.미션 서드', 'Open Food',
       '그릴드 비프 샐러드', '까르보나라', '모둠 해산물 플래터', '미션 서드 카베르네 

# 1. menu_category     
어떤 카테고리에 속하는지 분류         

ex.     
['main', 'side', 'drink', 'beverage', 'other']     
['육류', '생선류', '채소류']     
['밥', '빵', '면']   

In [3]:
df['menu_category'] = df['메뉴명'].map(menu_category)

# 2. avg_sales_all_days
자주 꾸준히 팔리는 메뉴인지 판단. 값이 높을수록 기본 수요가 큼

총 매출수량 ÷ 전체 일수 (= 총 매출수량 / 총 날짜 수)

In [4]:
menu_avg = df.groupby('메뉴명')['매출수량'].mean()
df['avg_sales_all_days'] = df['메뉴명'].map(menu_avg)

# 3. avg_sales_nonzero_days
한 번 팔릴 때 얼마나 많이 팔리는지를 나타냄. 단체주문 메뉴일 가능성 있음

총 매출수량 ÷ 매출수량이 0이 아닌 날 수

In [5]:
nonzero_avg = df[df['매출수량'] > 0].groupby('메뉴명')['매출수량'].mean()
df['avg_sales_nonzero_days'] = df['메뉴명'].map(nonzero_avg)
df

,영업일자,영업장명_메뉴명,매출수량,연,월,일,영업장,메뉴명,요일,is_holiday,menu_category,avg_sales_all_days,avg_sales_nonzero_days
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,2023,1,1,느티나무 셀프BBQ,1인 수저세트,일,1,기타,5.088346,9.432056
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,2023,1,2,느티나무 셀프BBQ,1인 수저세트,월,0,기타,5.088346,9.432056
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,2023,1,3,느티나무 셀프BBQ,1인 수저세트,화,0,기타,5.088346,9.432056
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2023,1,4,느티나무 셀프BBQ,1인 수저세트,수,0,기타,5.088346,9.432056
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,2023,1,5,느티나무 셀프BBQ,1인 수저세트,목,0,기타,5.088346,9.432056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12,2024,6,11,화담숲카페,현미뻥스크림,화,0,디저트,20.270677,38.931408
102672,2024-06-12,화담숲카페_현미뻥스크림,10,2024,6,12,화담숲카페,현미뻥스크림,수,0,디저트,20.270677,38.931408
102673,2024-06-13,화담숲카페_현미뻥스크림,14,2024,6,13,화담숲카페,현미뻥스크림,목,0,디저트,20.270677,38.931408
102674,2024-06-14,화담숲카페_현미뻥스크림,12,2024,6,14,화담숲카페,현미뻥스크림,금,0,디저트,20.270677,38.931408


# 4. zero_sales_day_ratio
비율이 높을수록 안 팔리는 날이 많음 (비정기적 메뉴일 수 있음)

(매출수량 0인 날 수 ÷ 전체 날짜 수) × 100

In [6]:
zero_ratio = df.groupby('메뉴명')['매출수량'].apply(lambda x: (x.eq(0).sum() / len(x)) * 100)

df['zero_sales_day_ratio'] = df['메뉴명'].map(zero_ratio)
df

,영업일자,영업장명_메뉴명,매출수량,연,월,일,영업장,메뉴명,요일,is_holiday,menu_category,avg_sales_all_days,avg_sales_nonzero_days,zero_sales_day_ratio
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,2023,1,1,느티나무 셀프BBQ,1인 수저세트,일,1,기타,5.088346,9.432056,46.052632
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,2023,1,2,느티나무 셀프BBQ,1인 수저세트,월,0,기타,5.088346,9.432056,46.052632
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,2023,1,3,느티나무 셀프BBQ,1인 수저세트,화,0,기타,5.088346,9.432056,46.052632
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2023,1,4,느티나무 셀프BBQ,1인 수저세트,수,0,기타,5.088346,9.432056,46.052632
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,2023,1,5,느티나무 셀프BBQ,1인 수저세트,목,0,기타,5.088346,9.432056,46.052632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12,2024,6,11,화담숲카페,현미뻥스크림,화,0,디저트,20.270677,38.931408,47.932331
102672,2024-06-12,화담숲카페_현미뻥스크림,10,2024,6,12,화담숲카페,현미뻥스크림,수,0,디저트,20.270677,38.931408,47.932331
102673,2024-06-13,화담숲카페_현미뻥스크림,14,2024,6,13,화담숲카페,현미뻥스크림,목,0,디저트,20.270677,38.931408,47.932331
102674,2024-06-14,화담숲카페_현미뻥스크림,12,2024,6,14,화담숲카페,현미뻥스크림,금,0,디저트,20.270677,38.931408,47.932331


# 5. is_sparse_menu
자주 팔리진 않지만 간헐적으로 팔리는 메뉴 여부

zero_sales_day_ratio > 50 이면 1, 아니면 0

In [7]:
df['is_sparse_menu'] = np.where(df['zero_sales_day_ratio'] > 50, 1, 0)
df

,영업일자,영업장명_메뉴명,매출수량,연,월,일,영업장,메뉴명,요일,is_holiday,menu_category,avg_sales_all_days,avg_sales_nonzero_days,zero_sales_day_ratio,is_sparse_menu
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,2023,1,1,느티나무 셀프BBQ,1인 수저세트,일,1,기타,5.088346,9.432056,46.052632,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,2023,1,2,느티나무 셀프BBQ,1인 수저세트,월,0,기타,5.088346,9.432056,46.052632,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,2023,1,3,느티나무 셀프BBQ,1인 수저세트,화,0,기타,5.088346,9.432056,46.052632,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2023,1,4,느티나무 셀프BBQ,1인 수저세트,수,0,기타,5.088346,9.432056,46.052632,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,2023,1,5,느티나무 셀프BBQ,1인 수저세트,목,0,기타,5.088346,9.432056,46.052632,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12,2024,6,11,화담숲카페,현미뻥스크림,화,0,디저트,20.270677,38.931408,47.932331,0
102672,2024-06-12,화담숲카페_현미뻥스크림,10,2024,6,12,화담숲카페,현미뻥스크림,수,0,디저트,20.270677,38.931408,47.932331,0
102673,2024-06-13,화담숲카페_현미뻥스크림,14,2024,6,13,화담숲카페,현미뻥스크림,목,0,디저트,20.270677,38.931408,47.932331,0
102674,2024-06-14,화담숲카페_현미뻥스크림,12,2024,6,14,화담숲카페,현미뻥스크림,금,0,디저트,20.270677,38.931408,47.932331,0


# 6. check

In [ ]:
# 예: 전처리 완료 데이터프레임이 df라면
df.to_csv('../data/train/train_processed.csv', index=False, encoding='utf-8-sig')